## Bert와 같ㅌ은 거대한 모델을 학습시키기 위해  매우 작은 데이터셋 사용가능

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00


In [2]:
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM

In [3]:
# Load the pretrained RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)

In [4]:
tokenizer

RobertaTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [5]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [6]:
# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
device

device(type='cpu')

In [8]:
# Set the model to evaluation mode
model.eval()

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [9]:
# Input text with a masked token
input_text = "The capital of Korea is <mask>."

In [10]:
# Tokenize input text
input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors='pt').to(device)

In [11]:
input_ids

tensor([[    0,   133,   812,     9,  1101,    16, 50264,     4,     2]])

In [27]:
# Masked language modeling
with torch.no_grad():
    outputs = model.to(device)(input_ids)
    predictions = outputs[0]
    print(outputs[0])
    print((outputs[0].shape))
## shape가 (1,9,50265 ) =
## 즉 들어온 모든 text embedding 값에 대해 각기 확률값존재

# Get the predicted token
# print(torch.where(input_ids == tokenizer.mask_token_id)) => 0.6
## masked token Id=0은 sos 인듯
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
## 즉 전체의 vocab_size==embedding size에서 masking된 값을 argmax를 통해서 알아낸다.
print(len((predictions[0, masked_index].squeeze())),predictions[0, masked_index])
predicted_token_ids = torch.argmax(predictions[0, masked_index], dim=1).tolist()
## 1,9,50265 에서 masked token의 위치인 모든 확률값을 가져오고 그중 가장 확률값이높게 생성되는 것들 가져온다.
predicted_tokens = tokenizer.batch_decode(predicted_token_ids)

# Print the predicted token
print("Predicted token:", predicted_tokens)

tensor([[[33.6948, -3.9591, 18.9807,  ...,  3.0111,  5.2757, 11.6480],
         [ 7.8751, -2.9956, 19.6839,  ...,  3.1899,  4.2424,  8.7384],
         [-3.2596, -4.3450,  9.9631,  ..., -1.1315, -1.6988, -0.0456],
         ...,
         [-3.4669, -4.0814, 10.7547,  ..., -2.8022, -5.1995,  1.4774],
         [19.5788, -4.5430, 20.3826,  ...,  1.1609,  3.1616,  7.6659],
         [10.6032, -3.7698, 31.6309,  ...,  1.7999, -0.5655,  8.6553]]])
torch.Size([1, 9, 50265])
50265 tensor([[-3.4669, -4.0814, 10.7547,  ..., -2.8022, -5.1995,  1.4774]])
Predicted token: [' Seoul']
